In [2]:
import re
import csv
from tqdm import tqdm
import pandas as pd
import json
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

C:\Users\chaha\AppData\Local\Temp\ipykernel_32568\984645584.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
data = []
with open("./info/booksummaries.txt", 'r') as f:
  reader = csv.reader(f, dialect='excel-tab')
  for row in tqdm(reader):
    data.append(row)


0it [00:00, ?it/s]

16559it [00:00, 50947.62it/s]


In [4]:
book_id = []
book_name = []
summary = []
genre = []
# Iterate over the rows in the data

for i in tqdm(data):
  # Extract the information for each column and store it in the corresponding list
    book_id.append(i[0])
    book_name.append(i[2])
    genre.append(i[5])
    summary.append(i[6])

# Create a Pandas DataFrame from the lists
books = pd.DataFrame({'book_id': book_id, 'book_name': book_name,
                       'genre': genre, 'summary': summary})
books.head(10)

100%|██████████| 16559/16559 [00:00<00:00, 1284272.93it/s]


,book_id,book_name,genre,summary
0,620,Animal Farm,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,843,A Clockwork Orange,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,986,The Plague,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,1756,An Enquiry Concerning Human Understanding,,The argument of the Enquiry proceeds by a ser...
4,2080,A Fire Upon the Deep,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...
5,2152,All Quiet on the Western Front,"{""/m/098tmk"": ""War novel"", ""/m/016lj8"": ""Roman...","The book tells the story of Paul Bäumer, a Ge..."
6,2890,A Wizard of Earthsea,"{""/m/0dwly"": ""Children's literature"", ""/m/01hm...","Ged is a young boy on Gont, one of the larger..."
7,2950,Anyone Can Whistle,,The story is set in an imaginary American tow...
8,4081,Blade Runner 3: Replicant Night,"{""/m/06n90"": ""Science Fiction"", ""/m/014dfn"": ""...","Living on Mars, Deckard is acting as a consul..."
9,4082,Blade Runner 2: The Edge of Human,"{""/m/06n90"": ""Science Fiction"", ""/m/014dfn"": ""...",Beginning several months after the events in ...


In [5]:
#preprocessing of the data and removing bad names and other werid info to get the bst results
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


lemmatizer = WordNetLemmatizer()

stop_word = set(stopwords.words('english'))

def proc(inp):
  inp = str(inp)
  text = inp.lower()
  text = re.sub(r'[^a-zA-Z]'," ",text)

  tokenized = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokenized]
  return " ".join(tokens)

def preprocess_dataframe(df, column_name):
    df[column_name] = df[column_name].apply(proc)
    return df

#now preprocss each column

books_clean = preprocess_dataframe(books, 'book_name')
books_clean = preprocess_dataframe(books, 'genre')
books_clean = preprocess_dataframe(books, 'summary')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\chaha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chaha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chaha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\chaha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
books_clean["book_info"] = books_clean["summary"] + " " + books_clean["genre"] 
#deleting the summary and genre columns
books_clean.drop(['summary','genre'],inplace=True, axis=1)
#dropping the book id column
books_clean.drop(['book_id'],inplace=True, axis=1)


In [7]:
#vctorize the data and then compare the matricies to itself.

#first vecotrize the words into numbers and then run the cosine similairty on the matrix agaisnt itself
tf = TfidfVectorizer(analyzer = "word", ngram_range=(1,2), min_df=0.0, stop_words='english')

tfidf_matrix = tf.fit_transform(books_clean['book_info'])

cosine_sim =  cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_sim)

[[1.         0.00867306 0.00708674 ... 0.00456184 0.00165874 0.00644459]
 [0.00867306 1.         0.01337266 ... 0.00480672 0.00265714 0.00822467]
 [0.00708674 0.01337266 1.         ... 0.00662154 0.00403892 0.01456101]
 ...
 [0.00456184 0.00480672 0.00662154 ... 1.         0.00257448 0.01060132]
 [0.00165874 0.00265714 0.00403892 ... 0.00257448 1.         0.00298281]
 [0.00644459 0.00822467 0.01456101 ... 0.01060132 0.00298281 1.        ]]


In [24]:
def get_recommendations(liked_books_indices, top_n=5):
    avg_similarity = cosine_sim[liked_books_indices].mean(axis=0)
    top_similar_indices = avg_similarity.argsort()[::-1][:top_n]
    recommended_books = [(books_clean.index[idx], books_clean.iloc[idx]['book_info']) for idx in top_similar_indices]
    recommended_books_filtered = []
    for book_id, book_info in recommended_books:
        if book_id not in liked_books_indices:
            recommended_books_filtered.append((book_id, book_info))
    
    return recommended_books_filtered

In [36]:
liked_books_indices = [0, 1, 2]  # Example book indices that the user has liked
for i in liked_books_indices:
    print(books_clean.iloc[i])


book_name                                          animal farm
book_info    old major the old boar on the manor farm call ...
Name: 0, dtype: object
book_name                                   a clockwork orange
book_info    alex a teenager living in near future england ...
Name: 1, dtype: object
book_name                                           the plague
book_info    the text of the plague is divided into five pa...
Name: 2, dtype: object


In [ ]:
recommended_books = get_recommendations(liked_books_indices)
print(recommended_books)

In [37]:
with open('tst.json','r') as f:
  r = json.load(f)
print(r)
r.user

{'userBooks': {'LfAJT0xxa4RKZs7DKTda2YiEUvA3': {'9781250129130': {'authors': 'S. Jae-Jones', 'description': "Devoting herself to her musical career six months after the events of Wintersong, Liesl struggles with her brother's cold withdrawal and her own inability to forget the austere young man who inspired her efforts.", 'image': 'http://books.google.com/books/content?id=rxJFDwAAQBAJ&printsec=frontcover&img=1&zoom=1&edge=curl&source=gbs_api', 'infoLink': 'http://books.google.com/books?id=rxJFDwAAQBAJ&dq=isbn:9781250129130&hl=&source=gbs_api', 'isbn13': '9781250129130', 'publishedDate': '2018-02-06', 'title': 'Shadowsong'}}}}


In [40]:
# Flatten the data
rows = []
for user_id, books in r['userBooks'].items():
    for isbn, details in r.items():
        row = details.copy()
        row['user_id'] = user_id
        row['isbn'] = isbn
        rows.append(row)



                        LfAJT0xxa4RKZs7DKTda2YiEUvA3  \
0  {'9781250129130': {'authors': 'S. Jae-Jones', ...   

                        user_id       isbn  
0  LfAJT0xxa4RKZs7DKTda2YiEUvA3  userBooks  


In [39]:
first_book_title = df.iloc[0]['title']


KeyError: 'title'